## Cómo llegamos aquí

Con el objetivo de lograr una etiquetación diversa y representativa de nuestras reseñas, aplicamos un proceso de vectorización, generación de embeddings y asignación de etiquetas utilizando similitud de coseno, tal como se detalla en el notebook "etiquetas_parte_1". Sin embargo, debido a restricciones de hardware del equipo utilizado, no fue posible procesar todo el conjunto de datos simultáneamente. Por ello, optamos por un enfoque escalonado:

Primera Iteración: Tomamos una muestra representativa del 2% del total de reseñas y la procesamos conforme al procedimiento establecido en "etiquetas_parte_1".

Preparación para la Segunda Iteración: Una vez finalizado el procesamiento de la primera muestra, se eliminaron esos registros del dataframe fuente para asegurar que no se procesaran de nuevo.

Segunda Iteración (Resto_1): Sobre el conjunto de datos resultante, que hemos denominado "resto_1", aplicamos el mismo procedimiento de etiquetado.

Preparación para la Tercera Iteración: Después de procesar "resto_1", se eliminaron esos registros del dataframe fuente.

Tercera Iteración (Resto_2): Finalmente, se procedió con el procesamiento de "resto_2", aplicando nuevamente el proceso de etiquetado.

Tras culminar estas tres etapas, consolidamos los resultados de las tres muestras y los exportamos en un único archivo CSV.

La principal razón para adoptar este enfoque por etapas fue técnica: el equipo en el que se realizó la ejecución tiene limitaciones en cuanto al rendimiento, y se arriesgaba a congelarse si se superaba cierto umbral en el uso de recursos. Esta metodología escalonada nos permitió optimizar el procesamiento sin comprometer la integridad del sistema.

## Qué hacemos aquí


El sistema autoLM de Google Cloud Platform sugiere que las categorías de etiquetado tengan al menos 100 registros para garantizar que el modelo resultante sea robusto y preciso. Esta recomendación se basa en la necesidad de que el modelo tenga suficiente diversidad y cantidad de ejemplos para cada categoría, lo que le permite aprender y generalizar con eficacia las características distintivas asociadas con cada etiqueta.

#### Motivo para adoptar esta recomendación:

Robustez del modelo: Modelos entrenados con una cantidad suficiente de datos para cada categoría son menos propensos a sobreajustarse (overfitting) a las características particulares de una muestra pequeña, lo que podría hacer que el modelo sea ineficaz con datos no vistos.

Confianza en las predicciones: Al tener una representación adecuada para cada categoría, se incrementa la confianza en que las predicciones del modelo reflejen las verdaderas características de los datos y no anomalías derivadas de una muestra insuficiente.

Mejor generalización: Con más ejemplos, el modelo tiene una mejor oportunidad de captar la variabilidad y las sutilezas asociadas con cada etiqueta.

#### Decisión de eliminar categorías con menos de 100 registros:

Basándonos en las recomendaciones de autoLM y los puntos anteriores, decidimos eliminar registros con categorías con menos de 100 ejemplos en nuestra muestra final. Esta decisión tiene múltiples beneficios:

Calidad sobre cantidad: Al centrarnos solo en las categorías con una cantidad adecuada de registros, aseguramos que las etiquetas utilizadas para entrenar el modelo sean representativas y robustas.

Evitar ruido y distracciones: Al eliminar categorías con escasos ejemplos, reducimos el riesgo de introducir ruido en el modelo, que podría resultar en predicciones imprecisas.

Optimización del tiempo y recursos: La formación de un modelo en categorías con escasos ejemplos podría requerir más tiempo y recursos para lograr una precisión aceptable, si es que se logra.


In [1]:
import pandas as pd
df = pd.read_csv("muestra_etiquetas_3.csv")

In [2]:
df.shape

(9800, 2)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comentario  9799 non-null   object
 1   categoria   9800 non-null   object
dtypes: object(2)
memory usage: 153.3+ KB


In [5]:
# Obtener el conteo de cada categoría desde el DataFrame actual
conteo_categorias = df['categoria'].value_counts()

# Identifica las categorías con menos de 100 registros
categorias_a_eliminar = conteo_categorias[conteo_categorias < 100].index.tolist()

# Filtra el DataFrame para excluir las categorías identificadas
df_final = df[~df['categoria'].isin(categorias_a_eliminar)]

# Verifica el conteo de las categorías después de la eliminación
print(df_final['categoria'].value_counts())


n_Ambiente                    2943
n_Valor                       1424
n_Confort                     1371
n_Suministros                 1231
n_Servicio                    1069
p_Ambiente                     848
n_Ubicacion                    190
p_Confort                      156
n_Experiencia_gastronomica     118
n_Mantenimiento                112
p_Valor                        103
Name: categoria, dtype: int64


In [7]:
df_final.shape

(9565, 2)

In [8]:
initial_nan_count = df_final['comentario'].isna().sum()
print("Número inicial de valores NaN en 'comentario':", initial_nan_count)

Número inicial de valores NaN en 'comentario': 0


In [9]:
df_final.head(5)

,comentario,categoria
0,my wife and i have stayed at a lot of hotels b...,n_Servicio
1,i brought my bf here for his birthday weekend ...,n_Suministros
2,do not stay here i am stuck here for the next ...,n_Suministros
3,i actually worked here in while in college go...,p_Ambiente
4,clean updated and affordable owner is super ni...,n_Ambiente


In [10]:
df_final.to_csv('muestra_etiquetas_final.csv', index=False, encoding='utf-8')

In [11]:
sp1 = pd.read_csv("muestra_etiquetas_final.csv")

In [12]:
initial_nan_count = sp1['comentario'].isna().sum()
print("Número inicial de valores NaN en 'comentario':", initial_nan_count)

Número inicial de valores NaN en 'comentario': 0


In [14]:
df_final.to_csv('reviews_etiquetas_final.csv', index=False, header=False)